<a href="https://colab.research.google.com/github/Taweilo/Adidas_theLook/blob/main/Adidas_SQL_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# All table & columns in the dataset

In [ ]:
-- Check all table with their cols

SELECT
    table_name,
    STRING_AGG((column_name), ', ') AS columns
FROM
    `bigquery-public-data.thelook_ecommerce`.INFORMATION_SCHEMA.COLUMNS
GROUP BY
    table_name;

Create CSV with complete data, only filter the brand that are interested

In [ ]:
-- All data

SELECT
    oi.id AS order_item_id,
    o.order_id,
    p.category,
    p.name AS product_name,
    brand,
    o.created_at,
    EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) AS year, -- create year
    EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) AS month, -- create month
    o.status,
    o.shipped_at,
    o.delivered_at,
    o.returned_at,
    oi.sale_price,
    p.cost,
    oi.user_id,
    u.gender,
    u.state,
    u.country
FROM
    `bigquery-public-data.thelook_ecommerce.orders` AS o
LEFT JOIN (
    SELECT
        id,
        order_id,
        user_id,
        product_id,
        sale_price
    FROM
        `bigquery-public-data.thelook_ecommerce.order_items` -- combine data from order_items
) AS oi
    ON o.order_id = oi.order_id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` AS u -- receive user profile
    ON oi.user_id = u.id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` AS p -- receive product info
    ON oi.product_id = p.id
WHERE
    brand IN ('adidas', 'Nike', 'PUMA', 'Under Armour', 'Reebok', 'New Balance') -- study sports brand
ORDER BY
    o.created_at DESC;

1. Overall sales performance
- Total sports market
- Adidas's performance at 2024 Jan / compare with its competitors
- Adidas's sales among countries

In [ ]:
-- total market size (Agg revenue) of the brand on the ecommerce in 2024, Jan

SELECT
    ROUND(SUM(oi.sale_price), 2) AS total_revenue
FROM
    `bigquery-public-data.thelook_ecommerce.orders` AS o
LEFT JOIN (
    SELECT
        id, order_id, user_id, product_id, sale_price
    FROM
        `bigquery-public-data.thelook_ecommerce.order_items`
) AS oi
    ON o.order_id = oi.order_id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
    ON oi.user_id = u.id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
    ON oi.product_id = p.id
WHERE
    EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) = 1
    AND status NOT IN ('Cancelled', 'Returned')
    AND brand IN ('adidas', 'Nike', 'PUMA', 'Under Armour','Reebok','New Balance');

In [ ]:
-- Sales KPI (Rev, Cost, gross prpfits, mkt%) compare with its competitor

SELECT
    CONCAT(UPPER(SUBSTR(brand, 1, 1)), LOWER(SUBSTR(brand, 2))) AS brand,
    ROUND(SUM(oi.sale_price), 2) AS total_revenue,
    ROUND(SUM(p.cost), 2) AS total_cost,
    ROUND(SUM(oi.sale_price) - SUM(p.cost), 2) AS gross_profit,
    (
        ROUND(SUM(oi.sale_price) / (
            SELECT
                ROUND(SUM(oi.sale_price), 2) AS total_revenue
            FROM
                `bigquery-public-data.thelook_ecommerce.orders` AS o
            LEFT JOIN (
                SELECT
                    id, order_id, user_id, product_id, sale_price
                FROM
                    `bigquery-public-data.thelook_ecommerce.order_items`
            ) AS oi
                ON o.order_id = oi.order_id
            LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
                ON oi.user_id = u.id
            LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
                ON oi.product_id = p.id
            WHERE
                EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) = 2024
                AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) = 1
                AND status NOT IN ('Cancelled', 'Returned')
                AND brand IN ('adidas', 'Nike', 'PUMA', 'Under Armour','Reebok','New Balance')
        ), 2)
    ) AS market_share
FROM
    `bigquery-public-data.thelook_ecommerce.orders` AS o
LEFT JOIN (
        SELECT
            id, order_id, user_id, product_id, sale_price
        FROM
            `bigquery-public-data.thelook_ecommerce.order_items`
    ) AS oi
    ON o.order_id = oi.order_id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
    ON oi.user_id = u.id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
    ON oi.product_id = p.id
WHERE
    EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) = 1
    AND status NOT IN ('Cancelled', 'Returned')
    AND brand IN ('adidas', 'Nike', 'PUMA', 'Under Armour', 'Reebok', 'New Balance')
GROUP BY
    brand
ORDER BY
    brand ASC;

In [ ]:
-- Adidas's sales in different countries

SELECT
    country,
    ROUND(SUM(oi.sale_price), 2) AS total_revenue,
    ROUND(SUM(p.cost), 2) AS total_cost,
    ROUND(SUM(oi.sale_price) - SUM(p.cost), 2) AS gross_profit
FROM
    `bigquery-public-data.thelook_ecommerce.orders` AS o
LEFT JOIN (
    SELECT
        id, order_id, user_id, product_id, sale_price
    FROM
        `bigquery-public-data.thelook_ecommerce.order_items`
) AS oi
    ON o.order_id = oi.order_id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
    ON oi.user_id = u.id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
    ON oi.product_id = p.id
WHERE
    EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) = 1
    AND status NOT IN ('Cancelled', 'Returned')
    AND brand = 'adidas'
GROUP BY
    country
ORDER BY
    total_revenue DESC;

2. Sales trend

In [ ]:
 -- Adidas's sales KPI trend

SELECT
    year,
    month,
    ROUND(SUM(oi.sale_price), 2) AS total_revenue,
    ROUND(SUM(p.cost), 2) AS total_cost,
    ROUND(SUM(oi.sale_price) - SUM(p.cost), 2) AS gross_profit
FROM
    (
        SELECT *,
            EXTRACT(YEAR FROM TIMESTAMP(created_at)) AS year,
            EXTRACT(MONTH FROM TIMESTAMP(created_at)) AS month
        FROM
            `bigquery-public-data.thelook_ecommerce.orders`
    ) AS o
LEFT JOIN (
    SELECT
        id, order_id, user_id, product_id, sale_price
    FROM
        `bigquery-public-data.thelook_ecommerce.order_items`
) AS oi
    ON o.order_id = oi.order_id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
    ON oi.user_id = u.id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
    ON oi.product_id = p.id
WHERE
     (
         ( EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) = 2023
      AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) IN (1,2,3,4,5,6,7,8,9,10,11,12))
  OR (EXTRACT(YEAR FROM TIMESTAMP(o.created_at)) = 2024
            AND EXTRACT(MONTH FROM TIMESTAMP(o.created_at)) = 1)
    )
    AND status NOT IN ('Cancelled', 'Returned')
    AND brand ='adidas'
GROUP BY
    year, month
ORDER BY
    year DESC, month DESC;

3. product performance
- The most popular product, category
- The most cancelled product, category
- The most returned product, category

In [ ]:
-- TOP 20 most popular product on the platform

SELECT
    product_id,
    name,
    COUNT(oi.product_id) AS frequency,
    brand,
    category
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status NOT IN ('Cancelled', 'Returned')
    AND brand IN ('adidas', 'Nike', 'PUMA', 'Under Armour','Reebok','New Balance')
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 1
GROUP BY
    name, product_id, brand, category
ORDER BY
    COUNT(oi.product_id) DESC
LIMIT 20;


In [ ]:
-- Adidas's top 10 popular product

SELECT
    product_id,
    name,
    COUNT(oi.product_id) AS purchase_frequency,
    category
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status NOT IN ('Cancelled', 'Returned')
    AND brand = 'adidas'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
GROUP BY
    name, product_id, category
ORDER BY
    COUNT(oi.product_id) DESC
LIMIT 10;

In [ ]:
-- top 10 Adidas's cancelled product

SELECT
    product_id,
    name,
    COUNT(oi.product_id) AS cancel_frequency,
    category
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status = 'Cancelled'
    AND brand = 'adidas'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
GROUP BY
    name, product_id, category
ORDER BY
    COUNT(oi.product_id) DESC
LIMIT 10;

In [ ]:
-- top 10 Adidas's returned product

SELECT
    product_id,
    name,
    COUNT(oi.product_id) AS returned_frequency,
    category
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status = 'Returned'
    AND brand = 'adidas'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
GROUP BY
    name, product_id, category
ORDER BY
    COUNT(oi.product_id) DESC
LIMIT 10;

4. Customer profile
- customer location
- age
- gender
- average spend
CLV, RMF?

In [ ]:
-- customers profile adidas's customer from which countries, age, gender

SELECT
    oi.order_id,
    u.id AS user_id,
    u.age,
    u.gender,
    u.country
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status NOT IN ('Returned', 'Cancelled')
    AND brand = 'adidas'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1;


In [ ]:
-- age profile of adidas'scustomer

SELECT
    age,
    COUNT(DISTINCT u.id) AS customer_number
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
INNER JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
INNER JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status NOT IN ('Returned', 'Cancelled')
    AND brand = 'adidas'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
GROUP BY
    age
ORDER BY
    age;


In [ ]:
SELECT
    age_group,
    COUNT(user_id) AS customer_count
FROM
(
SELECT
        user_id,
        age,
        CASE
            WHEN age BETWEEN 13 AND 19 THEN 'Teens'
            WHEN age BETWEEN 20 AND 35 THEN 'Young Adults'
            WHEN age BETWEEN 36 AND 50 THEN 'Adults'
            ELSE 'Seniors'
        END AS age_group
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
INNER JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
INNER JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status NOT IN ('Returned', 'Cancelled')
    AND brand = 'adidas'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
) AS grouped_customers
GROUP BY
    age_group
ORDER BY
    CASE age_group
        WHEN 'Teens' THEN 1
        WHEN 'Young Adults' THEN 2
        WHEN 'Adults' THEN 3
        WHEN 'Seniors' THEN 4
        ELSE 5  -- To handle any unexpected age groups
    END;

In [ ]:
-- country profile of adidas's customer

SELECT
    country,
    COUNT(DISTINCT u.id) AS customer_number
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status NOT IN ('Returned', 'Cancelled')
    AND brand = 'adidas'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
GROUP BY
    country
ORDER BY
    COUNT(DISTINCT u.id) DESC;


In [ ]:
-- gender profile of adidas'scustomer

SELECT
    gender,
    COUNT(DISTINCT u.id) AS customer_number
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status NOT IN ('Returned', 'Cancelled')
    AND brand = 'adidas'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
GROUP BY
    gender
ORDER BY
    gender;


In [ ]:
-- study the sales price with sales volume

SELECT
    ROUND(oi.sale_price, 2),
    COUNT(oi.id) AS sales_volume
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status NOT IN ('Returned', 'Cancelled')
    AND brand = 'adidas'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) IN (2023, 2024)
GROUP BY
    oi.sale_price
ORDER BY
    oi.sale_price;


In [ ]:
-- AOV, APF, Customer value

SELECT
    ROUND((SUM(sale_price) / COUNT(oi.id)), 2) AS aov, -- Revenue/#Order
    ROUND((COUNT(oi.id) / COUNT(DISTINCT u.id)), 2) AS apf, -- #Order/Unique users
    ROUND((SUM(sale_price) / COUNT(DISTINCT u.id)), 2) AS customer_value --aov*apf
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
WHERE
    oi.status NOT IN ('Cancelled', 'Returned')
    AND p.brand = 'adidas'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1;

In [2]:
-- Customer buying behavior

WITH rfm_data AS (
    SELECT
        oi.user_id,
        TIMESTAMP_DIFF('2024-01-31', MAX(oi.created_at), DAY) AS recency,
        COUNT(oi.id) AS frequency,
        SUM(oi.sale_price) AS monetary
    FROM
        `bigquery-public-data.thelook_ecommerce.order_items` AS oi
    LEFT JOIN
        `bigquery-public-data.thelook_ecommerce.products` AS p
    ON
        oi.product_id = p.id
    WHERE
        oi.status = 'Complete'
        AND p.brand = 'adidas'
        AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
        AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
    GROUP BY oi.user_id
)

SELECT
    ROUND(AVG(recency),2) AS Avg_recency,
    ROUND(AVG(frequency),2) AS Avg_frequency,
    ROUND(AVG(monetary),2) AS Avg_monetary

FROM
    rfm_data;


SyntaxError: invalid syntax (<ipython-input-2-4e31a39b56c2>, line 1)

In [ ]:
-- frm analysis of adidas's customer (2019~2024)

WITH rfm_data AS (
    SELECT
        oi.user_id,
        TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), MAX(oi.created_at), DAY) AS recency,
        COUNT(oi.id) AS frequency,
        SUM(oi.sale_price) AS monetary
    FROM
        `bigquery-public-data.thelook_ecommerce.order_items` AS oi
    LEFT JOIN
        `bigquery-public-data.thelook_ecommerce.products` AS p
    ON
        oi.product_id = p.id
    WHERE
        oi.status = 'Complete'
        AND p.brand = 'adidas'
        AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) IN (2019,2020,2021,2022,2023,2024)
    GROUP BY oi.user_id
)

SELECT
    user_id,
    recency,
    frequency,
    monetary,
    CASE
        WHEN recency <= 30 AND frequency >= 4 AND monetary >= 300 THEN 'Champions'
        WHEN recency <= 30 AND frequency >= 3 THEN 'Loyal Customers'
        WHEN recency <= 60 AND frequency >= 2 THEN 'Potential Loyalists'
        WHEN recency <= 90 THEN 'Recent Customers'
        ELSE 'Needs Attention'
    END AS rfm_segment
FROM
    rfm_data
ORDER BY frequency DESC;

In [ ]:
-- what else product do they purchase for adidas's customer
-- order with adidas's product and non-adidas product

WITH order_list AS (
    SELECT adidas_order.order_id AS order_id
    FROM (
        SELECT DISTINCT order_id
        FROM
            `bigquery-public-data.thelook_ecommerce.order_items` AS oit
        LEFT JOIN
            `bigquery-public-data.thelook_ecommerce.products` AS p
        ON
            oit.product_id = p.id
        WHERE
            status = 'Complete'
            AND brand = 'adidas'
    ) AS adidas_order
    INNER JOIN (
        SELECT DISTINCT order_id
        FROM
            `bigquery-public-data.thelook_ecommerce.order_items` AS oit
        LEFT JOIN
            `bigquery-public-data.thelook_ecommerce.products` AS p
        ON
            oit.product_id = p.id
        WHERE
            status = 'Complete'
            AND brand != 'adidas'

    ) AS non_adidas_order
    ON
        adidas_order.order_id = non_adidas_order.order_id
)
SELECT
    order_list.order_id,
    STRING_AGG(CAST(oi.id AS STRING), ", ") AS order_item_id,
    STRING_AGG(name, ", ") AS non_Adidas_purchase,
    STRING_AGG(brand, ", ") AS non_Adidas_brand,
    STRING_AGG(category, ", ") AS non_Adidas_category
FROM order_list
INNER JOIN
        `bigquery-public-data.thelook_ecommerce.order_items` AS oi
ON
    order_list.order_id = oi.order_id
INNER JOIN
        `bigquery-public-data.thelook_ecommerce.products` AS prod
ON
    oi.product_id = prod.id
WHERE
     EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
     AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
     AND brand != 'adidas'
GROUP BY order_list.order_id
ORDER BY order_list.order_id;


In [ ]:
-- redefine competitors/ purchase preference

WITH order_list AS (
    SELECT adidas_order.order_id AS order_id
    FROM (
        SELECT DISTINCT order_id
        FROM
            `bigquery-public-data.thelook_ecommerce.order_items` AS oit
        LEFT JOIN
            `bigquery-public-data.thelook_ecommerce.products` AS p
        ON
            oit.product_id = p.id
        WHERE
            status = 'Complete'
            AND brand = 'adidas'
    ) AS adidas_order
    INNER JOIN (
        SELECT DISTINCT order_id
        FROM
            `bigquery-public-data.thelook_ecommerce.order_items` AS oit
        LEFT JOIN
            `bigquery-public-data.thelook_ecommerce.products` AS p
        ON
            oit.product_id = p.id
        WHERE
            status = 'Complete'
            AND brand != 'adidas'

    ) AS non_adidas_order
    ON
        adidas_order.order_id = non_adidas_order.order_id
)
SELECT
    brand,
    COUNT(brand) AS number_of_purchase

FROM order_list
INNER JOIN
        `bigquery-public-data.thelook_ecommerce.order_items` AS oi
ON
    order_list.order_id = oi.order_id
INNER JOIN
        `bigquery-public-data.thelook_ecommerce.products` AS prod
ON
    oi.product_id = prod.id
WHERE
     EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
     AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
     AND brand != 'adidas'
GROUP BY
     brand
ORDER BY
number_of_purchase DESC;

In [ ]:
-- Analyze adidas's customer's purchase behavior

SELECT
    adidas_user.user_id,
    adidas_user.product_name,
    STRING_AGG(prod.name, ", ") AS non_Adidas_purchase,
    STRING_AGG(brand, ", ") AS non_Adidas_brand
FROM (
    SELECT
        user_id,
        name AS product_name
    FROM
        `bigquery-public-data.thelook_ecommerce.order_items` AS oi
    LEFT JOIN
        `bigquery-public-data.thelook_ecommerce.users` AS u
    ON
        oi.user_id = u.id
    LEFT JOIN
        `bigquery-public-data.thelook_ecommerce.products` AS p
    ON
        oi.product_id = p.id
    WHERE
        status = 'Complete'
        AND brand = 'adidas'
        AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
        AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1
) AS adidas_user
LEFT JOIN (
    SELECT
        *
    FROM
        `bigquery-public-data.thelook_ecommerce.order_items`
    WHERE
        status = 'Complete'
        AND EXTRACT(YEAR FROM TIMESTAMP(created_at)) IN (2023,2024)
) AS order_item
ON
    adidas_user.user_id = order_item.user_id
LEFT JOIN
        `bigquery-public-data.thelook_ecommerce.products` AS prod
ON
    order_item.product_id = prod.id
WHERE brand != 'adidas'
GROUP BY
    adidas_user.user_id,
    adidas_user.product_name
ORDER BY
    user_id;


5. Delivery efficency
- Lead time
- Delivery Duration

In [ ]:
-- Avg. lead time in 2024 Jan

SELECT
    ROUND((AVG(TIMESTAMP_DIFF(oi.delivered_at, oi.created_at, HOUR))/24), 2) AS avg_lead_time
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status IN ('Complete', 'Returned')
    AND brand = 'adidas'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1;

-- Avg. lead time in 2023

SELECT
    ROUND((AVG(TIMESTAMP_DIFF(oi.delivered_at, oi.created_at, HOUR))/24), 2) AS avg_lead_time
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status IN ('Complete', 'Returned')
    AND brand = 'adidas'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2023;

In [ ]:
-- Avg. shipping duration in 2024 Jan

SELECT
    ROUND((AVG(TIMESTAMP_DIFF(oi.delivered_at, oi.shipped_at, HOUR))/24), 2) AS avg_shipping_duration
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status IN ('Complete', 'Returned')
    AND brand = 'adidas'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1;

-- Avg. shipping duration in 2023

SELECT
    ROUND((AVG(TIMESTAMP_DIFF(oi.delivered_at, oi.shipped_at, HOUR))/24), 2) AS avg_shipping_duration
FROM
    `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS u
ON
    oi.user_id = u.id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    oi.product_id = p.id
WHERE
    status IN ('Complete', 'Returned')
    AND brand = 'adidas'
    AND EXTRACT(YEAR FROM TIMESTAMP(oi.created_at)) = 2023;


6. Marketing approach evaluation
- how many new customer at theLook / through what channel
- did they make the purchase / did they purchase adidas product

In [ ]:
-- new customer at theLook

SELECT
    id AS user_id,
    traffic_source,
    country,
    age,
    gender
FROM
    `bigquery-public-data.thelook_ecommerce.users`
WHERE
    EXTRACT(YEAR FROM TIMESTAMP(created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(oi.created_at)) = 1;

In [ ]:
-- channel traffic

SELECT
    traffic_source,
    COUNT(traffic_source) AS channel_traffic
FROM
    `bigquery-public-data.thelook_ecommerce.users`
WHERE
    EXTRACT(YEAR FROM TIMESTAMP(created_at)) = 2024
    AND EXTRACT(MONTH FROM TIMESTAMP(created_at)) = 1
GROUP BY
    traffic_source
ORDER BY
    channel_traffic DESC;


In [ ]:
-- new register who make a purchase + purchase frequency?

SELECT
    u.user_id,
    u.gender,
    u.age,
    p.name AS product_name,
    p.brand,
    p.category,
    p.retail_price
FROM (
    SELECT
        id AS user_id,
        *
    FROM
        `bigquery-public-data.thelook_ecommerce.users`
    WHERE
        EXTRACT(YEAR FROM TIMESTAMP(created_at)) = 2024
        AND EXTRACT(MONTH FROM TIMESTAMP(created_at)) = 1
) AS u
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.order_items` AS o
ON
    u.user_id = o.user_id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    o.product_id = p.id;


In [ ]:
-- new register brand purchase distribution

SELECT
    p.brand,
    COUNT(u.user_id) AS brand_purchase_by_new_customer,
    ROUND(SUM(o.sale_price),2) AS revenue,
    STRING_AGG(p.name, ', ') AS product
FROM (
    SELECT
        id AS user_id,
        *
    FROM
        `bigquery-public-data.thelook_ecommerce.users`
    WHERE
        EXTRACT(YEAR FROM TIMESTAMP(created_at)) = 2024
        AND EXTRACT(MONTH FROM TIMESTAMP(created_at)) = 1
) AS u
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.order_items` AS o
ON
    u.user_id = o.user_id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    o.product_id = p.id
WHERE
    p.brand IN ('adidas', 'Nike', 'PUMA', 'Under Armour', 'Reebok', 'New Balance')
GROUP BY p.brand
ORDER BY brand_purchase_by_new_customer DESC;


In [ ]:
-- new register who purchase adidas's product

SELECT
    u.user_id,
    u.gender,
    u.age,
    p.name AS product_name,
    p.brand,
    p.category,
    p.retail_price
FROM (
    SELECT
        id AS user_id,
        *
    FROM
        `bigquery-public-data.thelook_ecommerce.users`
    WHERE
        EXTRACT(YEAR FROM TIMESTAMP(created_at)) = 2024
        AND EXTRACT(MONTH FROM TIMESTAMP(created_at)) = 1
) AS u
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.order_items` AS o
ON
    u.user_id = o.user_id
LEFT JOIN
    `bigquery-public-data.thelook_ecommerce.products` AS p
ON
    o.product_id = p.id
WHERE
    p.brand = 'adidas';

Side proejct 01 RFM analysis
https://clevertap.com/blog/rfm-analysis/

Side project 02 - supply chain management
1. data governance
2. warehouse managment

In [ ]:
SELECT oi.id AS item_id,oi.delivered_at, oi.created_at,order_id,u.postal_code AS customer_postal_code,
u.longitude AS customer_longtitude,u.latitude AS customer_latitude,
ROUND(TIMESTAMP_DIFF(oi.delivered_at, oi.created_at, HOUR), 2) AS lead_time,
dc.name,dc.longitude AS dc_longitude,dc.latitude AS dc_latitude
FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi
LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
ON oi.user_id=u.id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.inventory_items` AS ii
ON oi.inventory_item_id=ii.id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.distribution_centers` AS dc
ON ii.product_distribution_center_id=dc.id
WHERE EXTRACT(YEAR FROM TIMESTAMP (oi.created_at)) = 2023
AND oi.status = 'Complete'
AND u.country = 'United States';